In [4]:
#seq2seq 모델
#
#두 개의 순환 신경망(rnn)로 이루어진 학습 모델
#- encoder : 입력값을 받아 이를 고정된 크기의 문맥 벡터(context vector)로 변환(읽기)
#- decoder : 문맥 벡터를 사용해서 출력 값 생성(쓰기)
#
#하나의 텍스트 문장이 입력으로 들어오면 하나의 텍스트 문장을 출력하는 구조, 생성 모델, 기계 번역에서 사용되는 모델
#
#원래 구글 번역기의 모델, 하지만 지금은 더 발전했음.

# 인코더에 입력이 있고 디코더에도 입력이 있다. 양쪽 모두 다 입력이 있음.
# 디코더의 입력과 인코더의 입력은 다르다.
# 인코더의 입력은 한번에 다준다. 우리가 아는대로 RNN을 거치면서 문장임베딩으로 바뀜
# 디코더의 입력은 하나만 준다. 하나씩도 아니고. 한개만 줌..왜,.?

# 쨌든 성능이 잘 안나와서 트랜스포머가 나옴.

In [5]:
#Transformer 
#
#- 순환 신경망의 한계
#
#이전 시점의 정보를 기억하고, 현재 시점의 입력과 함께 다음 시점의 출력을 생성하는 구조
#=> 앞을 보고 뒤를 맞추는 구조
#
#문장의 길이가 길수록 모든 정보를 하나의 벡터에 포함하기에 부족(앞부분 단어 정보 손실)
#
#입력 시퀀스의 순서에 따라 순차적으로 연산을 수행하기 때문에 병렬 처리가 어려움.
#
#이는 RNN의 학습과 추론 속도를 느리게 하고, 대규모 데이터와 모델을 다루기 어렵게 만듦.
#
#- Transformer
#
#2017년 구글이 발표한 논문(Attention is all you need)
#순환 신경망의 한계를 극복하기 위해 제안된 모델
#RNN을 사용하지 않고, 어텐션(attention) 알고리즘을 통해 시퀀스 데이터를 처리하는 모델

In [6]:
## 트랜스포머의 핵심은 self-attention
#
#입력 문장의 각 단어가 다른 단어들과 얼마나 관련이 있는지를 계산하고, 
#
#그 결과를 가중합하여 새로운 단어 임베딩 벡터를 생성하는 알고리즘
#
#과정1 : 입력 문장 -> 토큰화 + 임베딩 + positional encoding

# LSTM은 한단어씩 처리하니 순서가 명확한데 얘는 한꺼번에 다 넣고 처리한다. 단어에 족보가 없다. 순서가

# 가려지지 않는다. 그래서 임베딩에다가 position encoding 숫자 하나를 더 집어넣는다

### 위치정보가 반영된 임베딩 팩터를 만든다!! 

In [7]:
#- 과정 2 : 각 단어 임베딩 벡터 -> 쿼리(query), 키(key), 값(value) 벡터 생성
#
#쿼리(query), 키(key), 값(value) 벡터 간의 영상을 통해서 문맥적 관계성을 추출하는 과정

# 행렬 곱 --> '내적' 연속

#1) query : 문장 내 다른 단어들과 어떤 관계를 가지는지를 파악하고자 하는 특정 단어의 벡터

# 각 단어들만의 query벡터가 존재.

#2) key : query가 문장 내 어떤 단어와 유사도가 큰 지를 계산하는 데 사용되는 단어 벡터
#--> 입력 문장의 전체 단어 벡터
#

#나머지 단어의 key벡터와 존재. 





## Self-Attention

In [8]:
### 필요한 라이브러리 임폴트

import tensorflow as tf
import numpy as np

In [9]:
### 실습용 데이터 생성

'''
1. sentence = 'i love you'
2. 토큰화 --> [i, love, you]
3. 각 토큰 --> 512 차원의 임베딩 벡터로 변환 --> (3, 512) 임베딩 행렬 생성
4. 표준 정규 분포로부터 랜덤한 실수 샘플링 --> (3, 512) 임베딩 행렬에 해당하는 데이터 생성
'''

# 랜덤 시드 설정
tf.random.set_seed(0)

# (3, 512) 임베딩 행렬에 해당하는 데이터 생성
embedding_shape = (3, 512) #np.random.normal 하지만 텐소플로우에서는 넘파이배열을 쓰지 않음. (표준정규분포곡선만들기)
embeddings = tf.random.normal(shape=embedding_shape)

# 결과 확인하기

print(embeddings)

tf.Tensor(
[[ 1.5110626   0.42292204 -0.41969493 ... -1.1673577   0.777814
   0.58657396]
 [-0.13087033 -0.4497122   3.3774817  ...  0.2500961  -0.69026154
  -0.8148735 ]
 [ 0.40156764  0.3129424  -0.87114996 ... -0.3053926   0.18731174
  -1.6565207 ]], shape=(3, 512), dtype=float32)


In [10]:
### 가중치 행렬 W_q, W_k, W_v (Weights : query, key, value) 

'''
1. 가중치 행렬 W_q, W_k, W_v의 모양 : (512, 512)
2. 가중치 행렬의 초기 값 --> 랜덤한 실수로 설정
3. 표준 정규 분포로부터 랜덤한 실수 샘플링 --> (512, 512) 가중치 행렬에 해당하는 데이터 W_q, W_k, W_v 생성
'''

# (512, 512*3) 모양의 가중치 행렬 생성
weights_shape = (512, 512*3)
weights = tf.random.normal(shape=weights_shape)
print(f'생성된 가중치 행렬의 모양 : {tf.shape(weights)}')

# 쿼리를 만드는 가중치행렬값은 0~512
# 키를 만드는 가중치행렬값은 512~1024
# 밸류를 만드는 가중치행렬값은 1024~1536

# 가중치 행렬 --> 분할 --> 모양 : (512, 512)
W_q = weights[:, 0:512]
W_k = weights[:, 512:1024]
W_v = weights[:, 1024:1536]

print(f'가중치 행렬 W_q의 모양 : {tf.shape(W_q)}')

print('*'*80)

print(f'가중치 W_k의 모양 : {tf.shape(W_k)}')

print('*'*80)

print(f'가중치 W-v의 모양 : {tf.shape(W_v)}')

생성된 가중치 행렬의 모양 : [ 512 1536]
가중치 행렬 W_q의 모양 : [512 512]
********************************************************************************
가중치 W_k의 모양 : [512 512]
********************************************************************************
가중치 W-v의 모양 : [512 512]


In [11]:
print(weights)

tf.Tensor(
[[ 1.0668802   0.19454929 -0.53082895 ...  0.28124705 -0.41999227
  -1.731296  ]
 [-0.88284314  0.07581621  0.36982587 ... -0.98544943 -0.07025491
  -0.35464865]
 [-1.4630128   1.008406    0.00408987 ...  2.3250976  -1.2281883
  -0.36647767]
 ...
 [ 0.10821439 -0.23559454 -0.17931989 ... -0.1931868   0.06187644
   0.02346488]
 [ 0.17702705  1.1367483   0.44510984 ...  0.46767908  2.3448489
  -0.61274064]
 [-0.32561806  0.1451457  -0.6740147  ...  0.5818147  -1.1355666
   0.1629009 ]], shape=(512, 1536), dtype=float32)


In [13]:
### q, k, v 생성

'''
### 임베딩 행렬과 가중치 행렬곱 --> 각 단어의 q, k, v 백터 생성
'''

# 전체 단어의 qkv 행렬 생성(linear algebra, 선형대수)
qkv = tf.linalg.matmul(a=embeddings, b=weights)

# 각 단어의 q, k, v 추출
q = qkv[:, 0:512]
k = qkv[:, 512:1024]
v = qkv[:, 1024:]


# 결과 확인하기
print(f'생성된 qkv의 모양 : \n{tf.shape(qkv)}')

print('*'*80)

print(f'생성된 q 행렬의 모양 : \n{tf.shape(q)}')

print('*'*80)

print(f'생성된 k 행렬의 모양 : \n{tf.shape(k)}')

print('*'*80)

print(f'생성된 v 행렬의 모양 : \n{tf.shape(v)}')

# i love you (query) i love you (query) 한번에 표를 만들어서 한번에 때려박아서 진행.

생성된 qkv의 모양 : 
[   3 1536]
********************************************************************************
생성된 q 행렬의 모양 : 
[  3 512]
********************************************************************************
생성된 k 행렬의 모양 : 
[  3 512]
********************************************************************************
생성된 v 행렬의 모양 : 
[  3 512]


In [17]:
### scaled_dot_product_attention 함수 정의 
# 벡터간의 곱 = 내적 matrix multiply ( matmul ) 행렬간의 곱 = 벡터간의 곱
# 행렬을 구성하고 있는 성분 벡터,와 벡터간의 곱. 행렬곱이란 벡터간의 내적.
# 벡터 간의 내적은 a.b a*b로 표현되기 때문에 adotproduct.

'''
transpose(전치하다) ex) (3,512) (3,512) 안맞으니 바꿔줘야함 (3,512) (512,3) K의T승 -> 뒤집어라라는 뜻.
self-attention
과정 3 : Scaled Doct-Product Attention
1) attention score --> query 벡터와 key벡터의 내적을 통해 각 단어 간의 유사도 점수를 계산
2) scaling : attention score 값 / key 벡터의 차원 크기(열의숫자, 특성의숫자, feature numbers=512, 나중에 매개변수로
지정가능)의 제곱근 -> attention score 값의 크기를 조정 -> 숫자의 범위를 조정.
3) attention weight
	1 스케일링을 거친 attention score를 softmax 함수에 입력
	2 입력 문장의 각 단어가 다른 단어들과 얼마나 관련이 있는지를 0과 1사이의 숫자로 표현


대각선이 1이 안나오는 대신에 가장 큰 숫자다.

'''
# query와 key간의 전치를 통해서 내적을 구했다 까지. 내적을 구하면 Self-attention을 구할 수 있다.

#열심히 안해도됨이건. 그냥 예시임
num = np.array([512.0])
num = tf.reshape(num, [1,1])

tf.math.sqrt(num)

#def scaled_dot_product_attention(q, k, v):
#    

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[22.627417]])>